## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-25-18-53-41 +0000,nyt,"Israeli Hospital Strike Kills 20 in Gaza, Incl...",https://www.nytimes.com/2025/08/25/world/middl...
1,2025-08-25-18-47-54 +0000,nypost,What to know about screwworm amid US human cas...,https://nypost.com/health/what-to-know-about-s...
2,2025-08-25-18-45-00 +0000,wsj,Mexican Cartel Patriarch Pleads Guilty to U.S....,https://www.wsj.com/world/americas/mexican-car...
3,2025-08-25-18-39-33 +0000,nyt,U.N. Peacekeepers in Lebanon Face Uncertain Fu...,https://www.nytimes.com/2025/08/25/world/europ...
4,2025-08-25-18-32-47 +0000,startribune,Minneapolis mayoral candidate says a car was s...,https://www.startribune.com/minneapolis-mayora...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2338/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
120,trump,26
285,new,12
417,will,8
4,gaza,7
263,you,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
34,2025-08-25-17-11-00 +0000,wsj,President Trump intensified his tough-on-crime...,https://www.wsj.com/politics/policy/trump-exec...,76
71,2025-08-25-15-26-19 +0000,nypost,Trump signs executive order banning flag burni...,https://nypost.com/2025/08/25/us-news/trump-si...,57
66,2025-08-25-15-40-13 +0000,nypost,Keurig Dr Pepper to buy JDE Peet’s in $18B dea...,https://nypost.com/2025/08/25/business/keurig-...,56
117,2025-08-25-11-00-00 +0000,nypost,Trump to sign executive order ending cashless ...,https://nypost.com/2025/08/25/us-news/presiden...,55
122,2025-08-25-10-58-32 +0000,nypost,Trump warns ‘sloppy’ Chris Christie could be p...,https://nypost.com/2025/08/25/us-news/trump-ha...,53


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
34,76,2025-08-25-17-11-00 +0000,wsj,President Trump intensified his tough-on-crime...,https://www.wsj.com/politics/policy/trump-exec...
66,56,2025-08-25-15-40-13 +0000,nypost,Keurig Dr Pepper to buy JDE Peet’s in $18B dea...,https://nypost.com/2025/08/25/business/keurig-...
72,49,2025-08-25-15-21-00 +0000,wsj,Kilmar Abrego Garcia was arrested by immigrati...,https://www.wsj.com/us-news/law/kilmar-abrego-...
163,35,2025-08-25-01-56-00 +0000,wsj,Here’s what you need to know about the unusual...,https://www.wsj.com/politics/policy/texas-cali...
0,33,2025-08-25-18-53-41 +0000,nyt,"Israeli Hospital Strike Kills 20 in Gaza, Incl...",https://www.nytimes.com/2025/08/25/world/middl...
8,32,2025-08-25-18-23-56 +0000,nypost,Mexican drug lord ‘El Mayo’ dodges death penal...,https://nypost.com/2025/08/25/us-news/mexican-...
143,28,2025-08-25-08-53-39 +0000,nypost,US confirms nation’s first travel-associated h...,https://nypost.com/2025/08/25/us-news/us-confi...
64,27,2025-08-25-15-46-00 +0000,wsj,Tech executives including VC firm Andreessen H...,https://www.wsj.com/politics/silicon-valley-la...
122,27,2025-08-25-10-58-32 +0000,nypost,Trump warns ‘sloppy’ Chris Christie could be p...,https://nypost.com/2025/08/25/us-news/trump-ha...
83,26,2025-08-25-14-05-10 +0000,nypost,Teen girl murdered by asylum seeker while call...,https://nypost.com/2025/08/25/world-news/girl-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
